In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
path = "../data/data_data_10Mill.csv"

In [8]:
BUCKET_NAME = "wagon-ml-bizot-27"
PATH_INSIDE_BUCKET = "data/bq_complete.csv"
path2 = "gs://{}/{}".format(BUCKET_NAME, PATH_INSIDE_BUCKET)

In [3]:
%%time 
df1 = dd.read_csv(path)
df1.compute()

CPU times: user 17.4 s, sys: 3.28 s, total: 20.7 s
Wall time: 11.7 s


dropoff_latitude  dropoff_longitude  fare_amount  \
0              40.765108         -73.982276         9.70   
1              40.743610         -73.996048        44.04   
2              40.779390         -73.955594         5.50   
3              40.754786         -73.977562         6.50   
4              40.731135         -73.999713        13.70   
...                  ...                ...          ...   
502833         40.752927         -73.975255         6.10   
502834         40.761622         -73.960535         6.50   
502835         40.751085         -73.972445        16.50   
502836         40.765225         -73.997750         8.50   
502837         40.730273         -73.997951        19.50   

                              key  passenger_count            pickup_datetime  \
0       2011-03-19 20:14:11+00:00                1  2011-03-19 20:14:11+00:00   
1       2015-06-13 15:37:42+00:00                1  2015-06-13 15:37:42+00:00   
2       2013-04-30 23:07:08+00:00                1  2013-04-30 23:07:08+00:00   
3       2013-01-22 19:41:15+00:00                1  2013-01-22 19:41:15+00:00   
4       2009-06-07 10:25:00+00:00                1  2009-06-07 10:25:00+00:00   
...                           ...              ...                        ...   
502833  2009-02-18 07:15:00+00:00                2  2009-02-18 07:15:00+00:00   
502834  2012-06-28 21:03:00+00:00                1  2012-06-28 21:03:00+00:00   
502835  2012-02-14 13:05:00+00:00                2  2012-02-14 13:05:00+00:00   
502836  2013-03-23 13:14:00+00:00                1  2013-03-23 13:14:00+00:00   
502837  2013-08-15 08:29:37+00:00                3  2013-08-15 08:29:37+00:00   

        pickup_latitude  pickup_longitude  
0             40.747916        -74.008139  
1             40.774082        -73.874504  
2             40.768150        -73.955696  
3             40.757127        -73.986279  
4             40.771845        -73.950752  
...                 ...               ...  
502833        40.770138        -73.961473  
502834        40.742112        -74.004102  
502835        40.806408        -73.962877  
502836        40.755962        -73.993468  
502837        40.769449        -73.982763  

[10000000 rows x 8 columns]

In [4]:
%%time
df2 = pd.read_csv(path)

CPU times: user 12.7 s, sys: 1.29 s, total: 14 s
Wall time: 14.2 s


In [22]:
def clean_df(df, test=False):
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df

def clean_df_dask(df, test=False):
    df = df.dropna(how='any')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df

In [26]:
df1 = dd.read_csv(path)

In [27]:
df1 = clean_df_dask(df1)

In [24]:
df1 = df1.compute()

KeyboardInterrupt: 